In [1]:
import networkx as nx
from networkx import ego_graph

import torch.optim as optim
import argparse
import numpy as np
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv, GATConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

#from logger import Logger
from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import WebKB
from torch_geometric.loader import DataLoader

In [3]:
dataset = WebKB(root='/tmp/Cornell', name='Cornell',transform=T.ToSparseTensor())
data = dataset[0]
#data.adj_t = data.adj_t.to_symmetric()
#data.adj_t = data.adj_t.to_symmetric()
print(data)
#split_idx = dataset.get_idx_split()
#train_idx = split_idx['train'].to(device)

Processing...


Data(x=[183, 1703], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=298])


Done!


In [ ]:
train_index = np.where(data.train_mask)[0]
print(len(train_index))
valid_index = np.where(data.val_mask)[0]
print(len(valid_index))
test_index = np.where(data.test_mask)[0]
print(len(test_index))

# GAT

In [4]:
import torch


class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 3
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 1].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Valid: {result[:, 1].max():.2f}')
            print(f'  Final Train: {result[argmax, 0]:.2f}')
            print(f'   Final Test: {result[argmax, 2]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)

            best_results = []
            for r in result:
                train1 = r[:, 0].max().item()
                valid = r[:, 1].max().item()
                train2 = r[r[:, 1].argmax(), 0].item()
                test = r[r[:, 1].argmax(), 2].item()
                best_results.append((train1, valid, train2, test))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Valid: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')

In [5]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout,heads):
        super(GAT, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GATConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(GATConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GATConv(hidden_channels, out_channels))

        self.dropout = dropout
        self.heads=heads

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            #x=F.softmax(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x.log_softmax(dim=-1)


def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    #print(len(out))
    #print(data.y.squeeze(1)[train_idx])
    loss = F.nll_loss(out, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total

@torch.no_grad()
def test(model, data, train_idx,valid_idx,test_idx):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [14]:
idx=[data.train_mask[i][0] for i in range(183)]
train_index = np.where(idx)[0]
print(train_index)

[  0   2   4   8  11  13  15  19  21  26  32  35  37  39  41  44  45  46
  50  53  58  59  60  61  63  65  66  70  71  72  73  74  75  76  77  78
  83  85  86  88  89  92  94  95  96 101 103 104 105 107 108 110 113 115
 116 119 120 121 123 124 125 126 130 131 133 135 137 139 140 141 142 144
 145 146 148 152 153 156 161 168 170 171 173 177 179 181 182]


In [9]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 8, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.6, 'epochs': 100, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01}

    args = objectview(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    dataset = WebKB(root='/tmp/Cornell', name='Cornell',transform=T.ToSparseTensor())
    data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    model = GAT(data.num_features, args.hidden_channels, dataset.num_classes, args.num_layers,
                    args.dropout,args.heads)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 2.0106, Train: 17.24%, Valid: 10.17% Test: 16.22%
Run: 01, Epoch: 02, Loss: 1.6645, Train: 19.54%, Valid: 8.47% Test: 24.32%
Run: 01, Epoch: 03, Loss: 1.4760, Train: 27.59%, Valid: 10.17% Test: 29.73%
Run: 01, Epoch: 04, Loss: 1.4070, Train: 43.68%, Valid: 13.56% Test: 35.14%
Run: 01, Epoch: 05, Loss: 1.2313, Train: 57.47%, Valid: 22.03% Test: 37.84%
Run: 01, Epoch: 06, Loss: 1.2357, Train: 58.62%, Valid: 35.59% Test: 43.24%
Run: 01, Epoch: 07, Loss: 1.1778, Train: 66.67%, Valid: 38.98% Test: 40.54%
Run: 01, Epoch: 08, Loss: 1.1505, Train: 68.97%, Valid: 37.29% Test: 37.84%
Run: 01, Epoch: 09, Loss: 1.0929, Train: 65.52%, Valid: 37.29% Test: 35.14%
Run: 01, Epoch: 10, Loss: 1.1256, Train: 65.52%, Valid: 40.68% Test: 35.14%
Run: 01, Epoch: 11, Loss: 1.0740, Train: 67.82%, Valid: 40.68% Test: 40.54%
Run: 01, Epoch: 12, Loss: 1.0649, Train: 66.67%, Valid: 37.29% Test: 40.54%
Run: 01, Epoch: 13, Loss: 1.0884, Train: 68.97%, Valid: 37.29% Test: 35.14%
Run: 01, Epoc

Run: 02, Epoch: 75, Loss: 0.6560, Train: 90.80%, Valid: 45.76% Test: 35.14%
Run: 02, Epoch: 76, Loss: 0.5745, Train: 90.80%, Valid: 44.07% Test: 35.14%
Run: 02, Epoch: 77, Loss: 0.6558, Train: 90.80%, Valid: 44.07% Test: 35.14%
Run: 02, Epoch: 78, Loss: 0.6455, Train: 91.95%, Valid: 44.07% Test: 37.84%
Run: 02, Epoch: 79, Loss: 0.5729, Train: 91.95%, Valid: 44.07% Test: 37.84%
Run: 02, Epoch: 80, Loss: 0.6347, Train: 90.80%, Valid: 45.76% Test: 37.84%
Run: 02, Epoch: 81, Loss: 0.6343, Train: 91.95%, Valid: 45.76% Test: 37.84%
Run: 02, Epoch: 82, Loss: 0.5630, Train: 91.95%, Valid: 45.76% Test: 37.84%
Run: 02, Epoch: 83, Loss: 0.5941, Train: 91.95%, Valid: 45.76% Test: 37.84%
Run: 02, Epoch: 84, Loss: 0.6171, Train: 93.10%, Valid: 45.76% Test: 37.84%
Run: 02, Epoch: 85, Loss: 0.4813, Train: 93.10%, Valid: 44.07% Test: 32.43%
Run: 02, Epoch: 86, Loss: 0.5921, Train: 93.10%, Valid: 42.37% Test: 29.73%
Run: 02, Epoch: 87, Loss: 0.5743, Train: 94.25%, Valid: 42.37% Test: 29.73%
Run: 02, Epo

Run: 04, Epoch: 56, Loss: 0.4914, Train: 95.40%, Valid: 44.07% Test: 35.14%
Run: 04, Epoch: 57, Loss: 0.4622, Train: 96.55%, Valid: 44.07% Test: 35.14%
Run: 04, Epoch: 58, Loss: 0.4688, Train: 95.40%, Valid: 42.37% Test: 37.84%
Run: 04, Epoch: 59, Loss: 0.6187, Train: 94.25%, Valid: 40.68% Test: 37.84%
Run: 04, Epoch: 60, Loss: 0.4748, Train: 94.25%, Valid: 40.68% Test: 40.54%
Run: 04, Epoch: 61, Loss: 0.4588, Train: 94.25%, Valid: 40.68% Test: 43.24%
Run: 04, Epoch: 62, Loss: 0.5203, Train: 94.25%, Valid: 40.68% Test: 43.24%
Run: 04, Epoch: 63, Loss: 0.4499, Train: 94.25%, Valid: 42.37% Test: 45.95%
Run: 04, Epoch: 64, Loss: 0.4967, Train: 94.25%, Valid: 42.37% Test: 43.24%
Run: 04, Epoch: 65, Loss: 0.4896, Train: 94.25%, Valid: 42.37% Test: 43.24%
Run: 04, Epoch: 66, Loss: 0.5661, Train: 91.95%, Valid: 42.37% Test: 43.24%
Run: 04, Epoch: 67, Loss: 0.5676, Train: 93.10%, Valid: 42.37% Test: 43.24%
Run: 04, Epoch: 68, Loss: 0.4711, Train: 93.10%, Valid: 42.37% Test: 43.24%
Run: 04, Epo

Run: 06, Epoch: 28, Loss: 0.7210, Train: 88.51%, Valid: 47.46% Test: 40.54%
Run: 06, Epoch: 29, Loss: 0.6531, Train: 88.51%, Valid: 47.46% Test: 40.54%
Run: 06, Epoch: 30, Loss: 0.6320, Train: 89.66%, Valid: 47.46% Test: 40.54%
Run: 06, Epoch: 31, Loss: 0.5699, Train: 89.66%, Valid: 45.76% Test: 40.54%
Run: 06, Epoch: 32, Loss: 0.6397, Train: 90.80%, Valid: 45.76% Test: 40.54%
Run: 06, Epoch: 33, Loss: 0.5767, Train: 90.80%, Valid: 45.76% Test: 40.54%
Run: 06, Epoch: 34, Loss: 0.5367, Train: 90.80%, Valid: 44.07% Test: 40.54%
Run: 06, Epoch: 35, Loss: 0.5383, Train: 90.80%, Valid: 44.07% Test: 40.54%
Run: 06, Epoch: 36, Loss: 0.5187, Train: 90.80%, Valid: 44.07% Test: 40.54%
Run: 06, Epoch: 37, Loss: 0.4449, Train: 91.95%, Valid: 44.07% Test: 40.54%
Run: 06, Epoch: 38, Loss: 0.5430, Train: 91.95%, Valid: 47.46% Test: 40.54%
Run: 06, Epoch: 39, Loss: 0.4873, Train: 91.95%, Valid: 49.15% Test: 40.54%
Run: 06, Epoch: 40, Loss: 0.5897, Train: 91.95%, Valid: 49.15% Test: 40.54%
Run: 06, Epo

Run: 08, Epoch: 02, Loss: 1.5552, Train: 42.53%, Valid: 25.42% Test: 32.43%
Run: 08, Epoch: 03, Loss: 1.2935, Train: 43.68%, Valid: 28.81% Test: 29.73%
Run: 08, Epoch: 04, Loss: 1.2018, Train: 52.87%, Valid: 28.81% Test: 32.43%
Run: 08, Epoch: 05, Loss: 1.2942, Train: 52.87%, Valid: 30.51% Test: 24.32%
Run: 08, Epoch: 06, Loss: 1.1977, Train: 49.43%, Valid: 20.34% Test: 27.03%
Run: 08, Epoch: 07, Loss: 1.1398, Train: 47.13%, Valid: 22.03% Test: 21.62%
Run: 08, Epoch: 08, Loss: 1.1963, Train: 44.83%, Valid: 25.42% Test: 21.62%
Run: 08, Epoch: 09, Loss: 1.2362, Train: 41.38%, Valid: 23.73% Test: 21.62%
Run: 08, Epoch: 10, Loss: 1.1072, Train: 45.98%, Valid: 28.81% Test: 32.43%
Run: 08, Epoch: 11, Loss: 1.0405, Train: 51.72%, Valid: 33.90% Test: 35.14%
Run: 08, Epoch: 12, Loss: 1.0024, Train: 59.77%, Valid: 44.07% Test: 37.84%
Run: 08, Epoch: 13, Loss: 1.0834, Train: 65.52%, Valid: 49.15% Test: 40.54%
Run: 08, Epoch: 14, Loss: 1.0544, Train: 71.26%, Valid: 50.85% Test: 43.24%
Run: 08, Epo

Run: 09, Epoch: 86, Loss: 0.4400, Train: 91.95%, Valid: 57.63% Test: 32.43%
Run: 09, Epoch: 87, Loss: 0.3701, Train: 91.95%, Valid: 59.32% Test: 35.14%
Run: 09, Epoch: 88, Loss: 0.4195, Train: 93.10%, Valid: 59.32% Test: 35.14%
Run: 09, Epoch: 89, Loss: 0.4366, Train: 90.80%, Valid: 59.32% Test: 35.14%
Run: 09, Epoch: 90, Loss: 0.3972, Train: 90.80%, Valid: 59.32% Test: 35.14%
Run: 09, Epoch: 91, Loss: 0.4220, Train: 91.95%, Valid: 55.93% Test: 35.14%
Run: 09, Epoch: 92, Loss: 0.4978, Train: 91.95%, Valid: 57.63% Test: 35.14%
Run: 09, Epoch: 93, Loss: 0.4399, Train: 90.80%, Valid: 57.63% Test: 35.14%
Run: 09, Epoch: 94, Loss: 0.3933, Train: 90.80%, Valid: 55.93% Test: 35.14%
Run: 09, Epoch: 95, Loss: 0.3503, Train: 90.80%, Valid: 55.93% Test: 35.14%
Run: 09, Epoch: 96, Loss: 0.5165, Train: 91.95%, Valid: 55.93% Test: 35.14%
Run: 09, Epoch: 97, Loss: 0.4682, Train: 93.10%, Valid: 55.93% Test: 35.14%
Run: 09, Epoch: 98, Loss: 0.3912, Train: 94.25%, Valid: 57.63% Test: 35.14%
Run: 09, Epo

# WISE Embedding

In [10]:
dataset = WebKB(root='/tmp/Cornell', name='Cornell',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[183, 1703], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=298])


In [11]:
import pandas as pd
Domain_Fec=pd.DataFrame(data.x.numpy())
label=pd.DataFrame(data.y.numpy(),columns =['class'])
Data=pd.concat([Domain_Fec,label], axis=1)
Data.head()

,0,1,2,3,4,5,6,7,8,9,...,1694,1695,1696,1697,1698,1699,1700,1701,1702,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [12]:
Number_nodes=len(data.y)
fe_len=len(data.x[0])
catagories=Data['class'].to_numpy()
data_by_class = {cls: Data.loc[Data['class'] == cls].drop(['class'], axis=1) for cls in range(max(catagories) + 1)}
basis = [[max(df[i]) for i in range(len(df.columns))] for df in data_by_class.values()]
sel_basis = [[int(list(df[i].to_numpy()).count(1) >= int(len(df[i].index)*0.1)) 
              for i in range(len(df.columns))]
             for df in data_by_class.values()]
feature_names = [ii for ii in range(fe_len)]

In [ ]:
#It takes long time
Fec=[]
for i in range(23):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=np.zeros(7)
    for j in range(1433):
        for i in range(max(catagories)+1):
            if f[j]==1 and basis[i][j]==1:
                count[i]=count[i]+1;

    for i in range(max(catagories)+1):
        vec.append(count[i])
    f.clear()
    Fec.append(vec)
print(Fec)

In [13]:
Fec=[]
for i in range(Number_nodes):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    for j in range(fe_len):
        if f[j]==1 and basis[0][j]==1:
            count=count+1;
        if f[j]==1 and basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and basis[4][j]==1:
            count4=count4+1;
    vec.append(count)
    vec.append(count1)
    vec.append(count2)
    vec.append(count3)
    vec.append(count4)
    #print(f)
    f.clear()
    Fec.append(vec)

In [ ]:
print(Fec)

In [14]:
SFec=[]
for i in range(Number_nodes):
    Svec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    for j in range(fe_len):
        if f[j]==1 and sel_basis[0][j]==1:
            count=count+1;
        if f[j]==1 and sel_basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and sel_basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and sel_basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and sel_basis[4][j]==1:
            count4=count4+1;
    Svec.append(count)
    Svec.append(count1)
    Svec.append(count2)
    Svec.append(count3)
    Svec.append(count4)
    #print(f)
    f.clear()
    SFec.append(Svec)

In [ ]:
print(SFec)

In [15]:
Inc_fe=torch.tensor(Fec)
sel_fe=torch.tensor(SFec)
CC_domain=torch.cat((Inc_fe, sel_fe), 1).float()
print(CC_domain)
CC_domain.type()

tensor([[  3.,   2.,   3.,  ...,   2.,   2.,   4.],
        [ 51.,  44.,  51.,  ...,  42.,  39.,  39.],
        [ 35.,  24.,  32.,  ...,  21.,  15.,  24.],
        ...,
        [ 69.,  61.,  79.,  ...,  63.,  55.,  59.],
        [ 88.,  85., 101.,  ...,  90.,  52.,  68.],
        [168., 144., 232.,  ..., 157.,  68., 115.]])


'torch.FloatTensor'

# W-GAT

In [21]:
data.x=CC_domain
print(data)

Data(x=[183, 10], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=557])


In [34]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 8, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 100, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = WebKB(root='/tmp/Texas', name='Texas',transform=T.ToSparseTensor())
    #data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    #idx_train=[data.train_mask[i][0] for i in range(len(data.y))]
    #train_idx = np.where(idx_train)[0]
    #idx_val=[data.val_mask[i][0] for i in range(len(data.y))]
    #valid_idx = np.where(idx_val)[0]
    #idx_test=[data.test_mask[i][0] for i in range(len(data.y))]
    #test_idx = np.where(idx_test)[0]
    
    model = GAT(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout,args.heads)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.9380, Train: 19.54%, Valid: 18.64% Test: 27.03%
Run: 01, Epoch: 02, Loss: 1.9017, Train: 19.54%, Valid: 18.64% Test: 27.03%
Run: 01, Epoch: 03, Loss: 1.7465, Train: 19.54%, Valid: 18.64% Test: 27.03%
Run: 01, Epoch: 04, Loss: 1.5494, Train: 20.69%, Valid: 18.64% Test: 27.03%
Run: 01, Epoch: 05, Loss: 1.4624, Train: 20.69%, Valid: 18.64% Test: 24.32%
Run: 01, Epoch: 06, Loss: 1.6378, Train: 19.54%, Valid: 18.64% Test: 24.32%
Run: 01, Epoch: 07, Loss: 1.5541, Train: 17.24%, Valid: 18.64% Test: 24.32%
Run: 01, Epoch: 08, Loss: 1.5098, Train: 19.54%, Valid: 20.34% Test: 29.73%
Run: 01, Epoch: 09, Loss: 1.5503, Train: 20.69%, Valid: 15.25% Test: 29.73%
Run: 01, Epoch: 10, Loss: 1.5107, Train: 24.14%, Valid: 16.95% Test: 29.73%
Run: 01, Epoch: 11, Loss: 1.5068, Train: 34.48%, Valid: 32.20% Test: 35.14%
Run: 01, Epoch: 12, Loss: 1.4287, Train: 41.38%, Valid: 42.37% Test: 37.84%
Run: 01, Epoch: 13, Loss: 1.4573, Train: 39.08%, Valid: 45.76% Test: 43.24%
Run: 01, Epo

Run: 02, Epoch: 81, Loss: 1.2336, Train: 44.83%, Valid: 54.24% Test: 40.54%
Run: 02, Epoch: 82, Loss: 1.1741, Train: 47.13%, Valid: 55.93% Test: 40.54%
Run: 02, Epoch: 83, Loss: 1.2064, Train: 49.43%, Valid: 54.24% Test: 40.54%
Run: 02, Epoch: 84, Loss: 1.1791, Train: 48.28%, Valid: 50.85% Test: 43.24%
Run: 02, Epoch: 85, Loss: 1.2359, Train: 48.28%, Valid: 47.46% Test: 43.24%
Run: 02, Epoch: 86, Loss: 1.1980, Train: 49.43%, Valid: 47.46% Test: 45.95%
Run: 02, Epoch: 87, Loss: 1.2249, Train: 50.57%, Valid: 47.46% Test: 45.95%
Run: 02, Epoch: 88, Loss: 1.2303, Train: 50.57%, Valid: 49.15% Test: 45.95%
Run: 02, Epoch: 89, Loss: 1.1805, Train: 51.72%, Valid: 49.15% Test: 45.95%
Run: 02, Epoch: 90, Loss: 1.2431, Train: 54.02%, Valid: 49.15% Test: 43.24%
Run: 02, Epoch: 91, Loss: 1.2024, Train: 52.87%, Valid: 54.24% Test: 43.24%
Run: 02, Epoch: 92, Loss: 1.0446, Train: 50.57%, Valid: 54.24% Test: 40.54%
Run: 02, Epoch: 93, Loss: 1.1965, Train: 50.57%, Valid: 55.93% Test: 35.14%
Run: 02, Epo

Run: 04, Epoch: 76, Loss: 1.0726, Train: 57.47%, Valid: 50.85% Test: 48.65%
Run: 04, Epoch: 77, Loss: 1.1059, Train: 57.47%, Valid: 50.85% Test: 48.65%
Run: 04, Epoch: 78, Loss: 1.0724, Train: 56.32%, Valid: 45.76% Test: 48.65%
Run: 04, Epoch: 79, Loss: 1.1021, Train: 54.02%, Valid: 44.07% Test: 48.65%
Run: 04, Epoch: 80, Loss: 1.1506, Train: 54.02%, Valid: 42.37% Test: 48.65%
Run: 04, Epoch: 81, Loss: 1.0146, Train: 54.02%, Valid: 42.37% Test: 48.65%
Run: 04, Epoch: 82, Loss: 1.0479, Train: 52.87%, Valid: 45.76% Test: 48.65%
Run: 04, Epoch: 83, Loss: 1.0096, Train: 52.87%, Valid: 45.76% Test: 48.65%
Run: 04, Epoch: 84, Loss: 1.0587, Train: 56.32%, Valid: 44.07% Test: 48.65%
Run: 04, Epoch: 85, Loss: 1.1019, Train: 56.32%, Valid: 45.76% Test: 48.65%
Run: 04, Epoch: 86, Loss: 0.9846, Train: 57.47%, Valid: 45.76% Test: 48.65%
Run: 04, Epoch: 87, Loss: 1.0478, Train: 59.77%, Valid: 45.76% Test: 48.65%
Run: 04, Epoch: 88, Loss: 1.1032, Train: 60.92%, Valid: 45.76% Test: 48.65%
Run: 04, Epo

Run: 06, Epoch: 73, Loss: 1.1401, Train: 54.02%, Valid: 44.07% Test: 51.35%
Run: 06, Epoch: 74, Loss: 1.0925, Train: 52.87%, Valid: 45.76% Test: 51.35%
Run: 06, Epoch: 75, Loss: 1.0769, Train: 54.02%, Valid: 45.76% Test: 51.35%
Run: 06, Epoch: 76, Loss: 1.1402, Train: 59.77%, Valid: 45.76% Test: 51.35%
Run: 06, Epoch: 77, Loss: 1.2157, Train: 60.92%, Valid: 45.76% Test: 59.46%
Run: 06, Epoch: 78, Loss: 1.0732, Train: 63.22%, Valid: 47.46% Test: 62.16%
Run: 06, Epoch: 79, Loss: 1.2159, Train: 65.52%, Valid: 44.07% Test: 59.46%
Run: 06, Epoch: 80, Loss: 1.1486, Train: 68.97%, Valid: 49.15% Test: 51.35%
Run: 06, Epoch: 81, Loss: 1.0983, Train: 68.97%, Valid: 50.85% Test: 45.95%
Run: 06, Epoch: 82, Loss: 1.0302, Train: 68.97%, Valid: 49.15% Test: 40.54%
Run: 06, Epoch: 83, Loss: 1.0482, Train: 63.22%, Valid: 50.85% Test: 37.84%
Run: 06, Epoch: 84, Loss: 1.0244, Train: 60.92%, Valid: 40.68% Test: 45.95%
Run: 06, Epoch: 85, Loss: 1.0369, Train: 60.92%, Valid: 42.37% Test: 48.65%
Run: 06, Epo

Run: 08, Epoch: 64, Loss: 1.1217, Train: 59.77%, Valid: 52.54% Test: 35.14%
Run: 08, Epoch: 65, Loss: 1.1921, Train: 57.47%, Valid: 50.85% Test: 37.84%
Run: 08, Epoch: 66, Loss: 1.1172, Train: 57.47%, Valid: 50.85% Test: 37.84%
Run: 08, Epoch: 67, Loss: 1.1422, Train: 56.32%, Valid: 50.85% Test: 35.14%
Run: 08, Epoch: 68, Loss: 1.1630, Train: 59.77%, Valid: 50.85% Test: 37.84%
Run: 08, Epoch: 69, Loss: 1.1467, Train: 57.47%, Valid: 52.54% Test: 40.54%
Run: 08, Epoch: 70, Loss: 1.1275, Train: 56.32%, Valid: 50.85% Test: 40.54%
Run: 08, Epoch: 71, Loss: 1.0506, Train: 56.32%, Valid: 50.85% Test: 37.84%
Run: 08, Epoch: 72, Loss: 1.1294, Train: 56.32%, Valid: 50.85% Test: 37.84%
Run: 08, Epoch: 73, Loss: 1.1057, Train: 56.32%, Valid: 52.54% Test: 40.54%
Run: 08, Epoch: 74, Loss: 1.0774, Train: 56.32%, Valid: 50.85% Test: 40.54%
Run: 08, Epoch: 75, Loss: 1.0634, Train: 56.32%, Valid: 52.54% Test: 43.24%
Run: 08, Epoch: 76, Loss: 1.1327, Train: 57.47%, Valid: 52.54% Test: 40.54%
Run: 08, Epo

Run: 10, Epoch: 54, Loss: 1.2316, Train: 51.72%, Valid: 54.24% Test: 51.35%
Run: 10, Epoch: 55, Loss: 1.2555, Train: 51.72%, Valid: 59.32% Test: 54.05%
Run: 10, Epoch: 56, Loss: 1.2319, Train: 51.72%, Valid: 61.02% Test: 54.05%
Run: 10, Epoch: 57, Loss: 1.3249, Train: 51.72%, Valid: 57.63% Test: 54.05%
Run: 10, Epoch: 58, Loss: 1.2839, Train: 51.72%, Valid: 57.63% Test: 56.76%
Run: 10, Epoch: 59, Loss: 1.3093, Train: 51.72%, Valid: 57.63% Test: 56.76%
Run: 10, Epoch: 60, Loss: 1.2719, Train: 51.72%, Valid: 57.63% Test: 56.76%
Run: 10, Epoch: 61, Loss: 1.2766, Train: 51.72%, Valid: 55.93% Test: 54.05%
Run: 10, Epoch: 62, Loss: 1.2508, Train: 51.72%, Valid: 55.93% Test: 54.05%
Run: 10, Epoch: 63, Loss: 1.2128, Train: 51.72%, Valid: 57.63% Test: 54.05%
Run: 10, Epoch: 64, Loss: 1.2203, Train: 51.72%, Valid: 59.32% Test: 54.05%
Run: 10, Epoch: 65, Loss: 1.2353, Train: 52.87%, Valid: 61.02% Test: 54.05%
Run: 10, Epoch: 66, Loss: 1.2441, Train: 50.57%, Valid: 64.41% Test: 48.65%
Run: 10, Epo

# Topological Embedding

In [35]:
dataset = WebKB(root='/tmp/Cornell', name='Cornell')
data = dataset[0]
print(data)

Data(x=[183, 1703], edge_index=[2, 298], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10])


In [36]:
Edge_idx=data.edge_index.numpy()
Node=range(Number_nodes)
Edgelist=[]
for i in range(len(Edge_idx[1])):
    Edgelist.append((Edge_idx[0][i],Edge_idx[1][i]))
#print(Edgelist)

In [37]:
# a "plain" graph is undirected
G = nx.DiGraph()

# give each a node a 'name', which is a letter in this case.
#G.add_node('a')

# the add_nodes_from method allows adding nodes from a sequence, in this case a list
#nodes_to_add = ['b', 'c', 'd']
G.add_nodes_from(Node)

# add edge from 'a' to 'b'
# since this graph is undirected, the order doesn't matter here
#G.add_edge('a', 'b')

# just like add_nodes_from, we can add edges from a sequence
# edges should be specified as 2-tuples
#edges_to_add = [('a', 'c'), ('b', 'c'), ('c', 'd')]
G.add_edges_from(Edgelist)


In [38]:
print(G.number_of_edges())

298


In [39]:
def Topological_Feature_subLevel(adj,filtration_fun, Filtration):
        betti_0=[]
        betti_1=[]
        for p in range(len(Filtration)):
            n_active = np.where(np.array(filtration_fun) <= Filtration[p])[0].tolist()
            Active_node=np.unique(n_active)
            if (len(Active_node)==0):
                betti_0.append(0)
                betti_1.append(0)
            else:
                b=adj[Active_node,:][:,Active_node]
                my_flag=pyflagser.flagser_unweighted(b, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                x = my_flag["betti"]
                betti_0.append(x[0])
                betti_1.append(x[1])
            n_active.clear()
        return betti_0,betti_1

In [40]:
def Degree_list(Graph):
    degree_list = [Graph.degree(node) for node in Graph.nodes]
    return np.array(degree_list)

In [41]:
degree_list=Degree_list(G)
unique_list=np.unique(degree_list)
for d in unique_list:
    count=0
    for i in range(len(degree_list)):
        if degree_list[i]==d:
            count=count+1
    print(int(d)," | ",count,'\n')

1  |  69 

2  |  40 

3  |  22 

4  |  20 

5  |  13 

6  |  4 

7  |  5 

8  |  3 

9  |  3 

10  |  2 

12  |  1 

94  |  1 



In [42]:
import pyflagser
Node_fil=[1,2,3,4,5,6,7,8,9,10,20,100]
topo_betti_0=[]
topo_betti_1=[]
Node_Edge=[]
for i in range(Number_nodes):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(Number_nodes-1)), end='', flush=True)
    subgraph=ego_graph(G, i, radius=2, center=True, undirected=True, distance=None)
    filt=Degree_list(subgraph)
    A_sub = nx.to_numpy_array(subgraph)# adjacency matrix of subgraph
    fe=Topological_Feature_subLevel(A_sub,filt,Node_fil)
    topo_betti_0.append(fe[0])
    topo_betti_1.append(fe[1])
    Node_Edge.append([subgraph.number_of_nodes(),subgraph.number_of_edges()])
    #topo_with_NE.app

Processing file 182 (100%)

In [44]:
dataset = WebKB(root='/tmp/Cornell', name='Cornell',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[183, 1703], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=298])


In [45]:
topo_betti0=torch.tensor(topo_betti_0).float()
topo_betti1=torch.tensor(topo_betti_1).float()
NodeEdge=torch.tensor(Node_Edge).float()

In [46]:
data.x=CC_domain
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

Data(x=[183, 10], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=298], topo=[183, 24])


# TOPO-W-GAT

In [47]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout,heads):
        super(GAT, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GATConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(GATConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GATConv(hidden_channels, out_channels))

        self.dropout = dropout
        self.heads=heads

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x
        #return x.log_softmax(dim=-1)

class MLP(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        #return torch.log_softmax(x, dim=-1)
        return x
    
class MLP2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP2, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp2(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.log_softmax(x, dim=-1)
    

def train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2):
    model.train()
    mlp_model.train()
    mlp_2.train()
    optimizer.zero_grad()
    optimizer_mlp.zero_grad()
    optimizer_mlp2.zero_grad()
    gcn_embedding = model(data.x, data.adj_t)[train_idx]
    #print(gcn_embedding)
    mlp_embedding = mlp_model(data.topo[train_idx])
    #print(mlp_embedding)
    combined_embedding = torch.cat((gcn_embedding, mlp_embedding), dim=1)
    #print(combined_embedding)
    mlp_emb = mlp_2(combined_embedding)
    #print(mlp_emb)
    loss = F.nll_loss(mlp_emb, data.y.squeeze()[train_idx])
    #loss = F.nll_loss(combined_embedding, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer_mlp2.step()
    optimizer.step()
    optimizer_mlp.step()
    

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total



@torch.no_grad()
def test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx):
    model.eval()
    mlp_model.eval()
    mlp_2.eval()

    gcn_out = model(data.x, data.adj_t)
    #print(gcn_out[0])
    mlp_out=mlp_model(data.topo)
    #print(mlp_out)
    #out=torch.cat((gcn_out,mlp_out),dim=1)
    Com=torch.cat((gcn_out,mlp_out),dim=1)
    out=mlp_2(Com)
    y_pred = out.argmax(dim=-1, keepdim=True)
    #print(y_pred[0])
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [54]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 8, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}
    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    model = GAT(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout,args.heads)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 10,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(20, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.001)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.6046, Train: 19.54%, Valid: 13.56% Test: 13.51%
Run: 01, Epoch: 02, Loss: 1.5460, Train: 41.38%, Valid: 49.15% Test: 43.24%
Run: 01, Epoch: 03, Loss: 1.5244, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 04, Loss: 1.4980, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 05, Loss: 1.4981, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 06, Loss: 1.4667, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 07, Loss: 1.4482, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 08, Loss: 1.4029, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 09, Loss: 1.4130, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 10, Loss: 1.3681, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 11, Loss: 1.3897, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 12, Loss: 1.3826, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 13, Loss: 1.3307, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epo

Run: 01, Epoch: 132, Loss: 0.8587, Train: 66.67%, Valid: 52.54% Test: 59.46%
Run: 01, Epoch: 133, Loss: 0.8653, Train: 67.82%, Valid: 52.54% Test: 59.46%
Run: 01, Epoch: 134, Loss: 0.9094, Train: 66.67%, Valid: 49.15% Test: 54.05%
Run: 01, Epoch: 135, Loss: 0.9933, Train: 66.67%, Valid: 49.15% Test: 51.35%
Run: 01, Epoch: 136, Loss: 0.8941, Train: 66.67%, Valid: 49.15% Test: 48.65%
Run: 01, Epoch: 137, Loss: 0.8737, Train: 67.82%, Valid: 47.46% Test: 48.65%
Run: 01, Epoch: 138, Loss: 1.0001, Train: 68.97%, Valid: 50.85% Test: 51.35%
Run: 01, Epoch: 139, Loss: 0.9972, Train: 67.82%, Valid: 52.54% Test: 48.65%
Run: 01, Epoch: 140, Loss: 0.8589, Train: 68.97%, Valid: 52.54% Test: 48.65%
Run: 01, Epoch: 141, Loss: 0.8786, Train: 66.67%, Valid: 55.93% Test: 51.35%
Run: 01, Epoch: 142, Loss: 0.9441, Train: 66.67%, Valid: 59.32% Test: 54.05%
Run: 01, Epoch: 143, Loss: 0.8325, Train: 66.67%, Valid: 59.32% Test: 59.46%
Run: 01, Epoch: 144, Loss: 0.9137, Train: 67.82%, Valid: 54.24% Test: 56.76%

Run: 02, Epoch: 66, Loss: 1.0820, Train: 62.07%, Valid: 49.15% Test: 40.54%
Run: 02, Epoch: 67, Loss: 1.0481, Train: 62.07%, Valid: 47.46% Test: 40.54%
Run: 02, Epoch: 68, Loss: 1.0487, Train: 60.92%, Valid: 47.46% Test: 37.84%
Run: 02, Epoch: 69, Loss: 1.1575, Train: 59.77%, Valid: 52.54% Test: 40.54%
Run: 02, Epoch: 70, Loss: 1.0932, Train: 54.02%, Valid: 52.54% Test: 40.54%
Run: 02, Epoch: 71, Loss: 1.0561, Train: 51.72%, Valid: 52.54% Test: 37.84%
Run: 02, Epoch: 72, Loss: 1.1407, Train: 52.87%, Valid: 54.24% Test: 45.95%
Run: 02, Epoch: 73, Loss: 1.0417, Train: 50.57%, Valid: 59.32% Test: 48.65%
Run: 02, Epoch: 74, Loss: 1.0687, Train: 52.87%, Valid: 54.24% Test: 43.24%
Run: 02, Epoch: 75, Loss: 1.0978, Train: 52.87%, Valid: 54.24% Test: 45.95%
Run: 02, Epoch: 76, Loss: 1.0981, Train: 55.17%, Valid: 54.24% Test: 40.54%
Run: 02, Epoch: 77, Loss: 1.1415, Train: 56.32%, Valid: 54.24% Test: 37.84%
Run: 02, Epoch: 78, Loss: 1.0799, Train: 55.17%, Valid: 55.93% Test: 37.84%
Run: 02, Epo

Run: 03, Epoch: 03, Loss: 1.5875, Train: 5.75%, Valid: 11.86% Test: 10.81%
Run: 03, Epoch: 04, Loss: 1.5632, Train: 43.68%, Valid: 33.90% Test: 51.35%
Run: 03, Epoch: 05, Loss: 1.4999, Train: 47.13%, Valid: 35.59% Test: 54.05%
Run: 03, Epoch: 06, Loss: 1.4619, Train: 47.13%, Valid: 35.59% Test: 54.05%
Run: 03, Epoch: 07, Loss: 1.4385, Train: 47.13%, Valid: 35.59% Test: 54.05%
Run: 03, Epoch: 08, Loss: 1.3956, Train: 47.13%, Valid: 35.59% Test: 54.05%
Run: 03, Epoch: 09, Loss: 1.3697, Train: 47.13%, Valid: 35.59% Test: 54.05%
Run: 03, Epoch: 10, Loss: 1.3802, Train: 47.13%, Valid: 35.59% Test: 54.05%
Run: 03, Epoch: 11, Loss: 1.3376, Train: 47.13%, Valid: 35.59% Test: 54.05%
Run: 03, Epoch: 12, Loss: 1.3140, Train: 47.13%, Valid: 35.59% Test: 54.05%
Run: 03, Epoch: 13, Loss: 1.3142, Train: 47.13%, Valid: 35.59% Test: 54.05%
Run: 03, Epoch: 14, Loss: 1.2759, Train: 47.13%, Valid: 35.59% Test: 54.05%
Run: 03, Epoch: 15, Loss: 1.2662, Train: 47.13%, Valid: 35.59% Test: 54.05%
Run: 03, Epoc

Run: 03, Epoch: 138, Loss: 0.8435, Train: 75.86%, Valid: 47.46% Test: 45.95%
Run: 03, Epoch: 139, Loss: 0.8389, Train: 75.86%, Valid: 45.76% Test: 45.95%
Run: 03, Epoch: 140, Loss: 0.7752, Train: 75.86%, Valid: 45.76% Test: 51.35%
Run: 03, Epoch: 141, Loss: 0.7489, Train: 74.71%, Valid: 47.46% Test: 54.05%
Run: 03, Epoch: 142, Loss: 0.7247, Train: 68.97%, Valid: 38.98% Test: 48.65%
Run: 03, Epoch: 143, Loss: 0.8004, Train: 59.77%, Valid: 38.98% Test: 45.95%
Run: 03, Epoch: 144, Loss: 0.7904, Train: 57.47%, Valid: 42.37% Test: 45.95%
Run: 03, Epoch: 145, Loss: 0.8276, Train: 60.92%, Valid: 38.98% Test: 45.95%
Run: 03, Epoch: 146, Loss: 0.8257, Train: 73.56%, Valid: 35.59% Test: 45.95%
Run: 03, Epoch: 147, Loss: 0.6834, Train: 81.61%, Valid: 45.76% Test: 54.05%
Run: 03, Epoch: 148, Loss: 0.8176, Train: 80.46%, Valid: 42.37% Test: 54.05%
Run: 03, Epoch: 149, Loss: 0.7763, Train: 80.46%, Valid: 45.76% Test: 51.35%
Run: 03, Epoch: 150, Loss: 0.7647, Train: 79.31%, Valid: 47.46% Test: 54.05%

...

Run: 08, Epoch: 65, Loss: 1.0974, Train: 50.57%, Valid: 52.54% Test: 40.54%
Run: 08, Epoch: 66, Loss: 1.1494, Train: 54.02%, Valid: 52.54% Test: 43.24%
Run: 08, Epoch: 67, Loss: 1.1529, Train: 54.02%, Valid: 54.24% Test: 43.24%
Run: 08, Epoch: 68, Loss: 1.1466, Train: 55.17%, Valid: 54.24% Test: 43.24%
Run: 08, Epoch: 69, Loss: 1.1209, Train: 56.32%, Valid: 57.63% Test: 43.24%
Run: 08, Epoch: 70, Loss: 1.1321, Train: 59.77%, Valid: 59.32% Test: 43.24%
Run: 08, Epoch: 71, Loss: 1.0740, Train: 62.07%, Valid: 62.71% Test: 43.24%
Run: 08, Epoch: 72, Loss: 1.0575, Train: 62.07%, Valid: 64.41% Test: 43.24%
Run: 08, Epoch: 73, Loss: 1.1016, Train: 64.37%, Valid: 62.71% Test: 43.24%
Run: 08, Epoch: 74, Loss: 1.1083, Train: 59.77%, Valid: 57.63% Test: 45.95%
Run: 08, Epoch: 75, Loss: 1.0524, Train: 62.07%, Valid: 57.63% Test: 43.24%
Run: 08, Epoch: 76, Loss: 1.1380, Train: 63.22%, Valid: 59.32% Test: 43.24%
Run: 08, Epoch: 77, Loss: 1.0781, Train: 62.07%, Valid: 61.02% Test: 40.54%
Run: 08, Epo

Run: 08, Epoch: 197, Loss: 0.9292, Train: 70.11%, Valid: 66.10% Test: 37.84%
Run: 08, Epoch: 198, Loss: 0.8931, Train: 70.11%, Valid: 66.10% Test: 37.84%
Run: 08, Epoch: 199, Loss: 0.9292, Train: 71.26%, Valid: 66.10% Test: 40.54%
Run: 08, Epoch: 200, Loss: 0.8454, Train: 71.26%, Valid: 66.10% Test: 37.84%
Run 08:
Highest Train: 74.71
Highest Valid: 74.58
  Final Train: 70.11
   Final Test: 48.65
Run: 09, Epoch: 01, Loss: 1.6571, Train: 25.29%, Valid: 15.25% Test: 18.92%
Run: 09, Epoch: 02, Loss: 1.6023, Train: 25.29%, Valid: 15.25% Test: 18.92%
Run: 09, Epoch: 03, Loss: 1.5712, Train: 25.29%, Valid: 15.25% Test: 18.92%
Run: 09, Epoch: 04, Loss: 1.5568, Train: 25.29%, Valid: 15.25% Test: 18.92%
Run: 09, Epoch: 05, Loss: 1.5171, Train: 25.29%, Valid: 15.25% Test: 18.92%
Run: 09, Epoch: 06, Loss: 1.5055, Train: 25.29%, Valid: 15.25% Test: 18.92%
Run: 09, Epoch: 07, Loss: 1.4414, Train: 25.29%, Valid: 15.25% Test: 18.92%
Run: 09, Epoch: 08, Loss: 1.4432, Train: 44.83%, Valid: 45.76% Test:

Run: 09, Epoch: 130, Loss: 0.6595, Train: 70.11%, Valid: 57.63% Test: 45.95%
Run: 09, Epoch: 131, Loss: 0.8212, Train: 70.11%, Valid: 59.32% Test: 54.05%
Run: 09, Epoch: 132, Loss: 0.8399, Train: 77.01%, Valid: 57.63% Test: 54.05%
Run: 09, Epoch: 133, Loss: 0.7178, Train: 63.22%, Valid: 54.24% Test: 48.65%
Run: 09, Epoch: 134, Loss: 0.7526, Train: 62.07%, Valid: 49.15% Test: 43.24%
Run: 09, Epoch: 135, Loss: 0.7253, Train: 63.22%, Valid: 55.93% Test: 45.95%
Run: 09, Epoch: 136, Loss: 0.6962, Train: 66.67%, Valid: 55.93% Test: 43.24%
Run: 09, Epoch: 137, Loss: 0.7004, Train: 77.01%, Valid: 61.02% Test: 48.65%
Run: 09, Epoch: 138, Loss: 0.7073, Train: 74.71%, Valid: 64.41% Test: 45.95%
Run: 09, Epoch: 139, Loss: 0.7939, Train: 66.67%, Valid: 55.93% Test: 51.35%
Run: 09, Epoch: 140, Loss: 0.7322, Train: 72.41%, Valid: 61.02% Test: 51.35%
Run: 09, Epoch: 141, Loss: 0.7540, Train: 51.72%, Valid: 67.80% Test: 35.14%
Run: 09, Epoch: 142, Loss: 0.7512, Train: 48.28%, Valid: 66.10% Test: 35.14%

Run: 10, Epoch: 64, Loss: 1.0841, Train: 55.17%, Valid: 64.41% Test: 51.35%
Run: 10, Epoch: 65, Loss: 1.1043, Train: 55.17%, Valid: 62.71% Test: 51.35%
Run: 10, Epoch: 66, Loss: 1.1057, Train: 57.47%, Valid: 62.71% Test: 48.65%
Run: 10, Epoch: 67, Loss: 1.0902, Train: 58.62%, Valid: 61.02% Test: 48.65%
Run: 10, Epoch: 68, Loss: 1.1053, Train: 57.47%, Valid: 62.71% Test: 54.05%
Run: 10, Epoch: 69, Loss: 1.0628, Train: 58.62%, Valid: 64.41% Test: 62.16%
Run: 10, Epoch: 70, Loss: 1.0771, Train: 60.92%, Valid: 62.71% Test: 59.46%
Run: 10, Epoch: 71, Loss: 1.0527, Train: 57.47%, Valid: 64.41% Test: 56.76%
Run: 10, Epoch: 72, Loss: 1.0890, Train: 55.17%, Valid: 64.41% Test: 56.76%
Run: 10, Epoch: 73, Loss: 1.0530, Train: 56.32%, Valid: 66.10% Test: 54.05%
Run: 10, Epoch: 74, Loss: 1.0436, Train: 56.32%, Valid: 62.71% Test: 54.05%
Run: 10, Epoch: 75, Loss: 1.0823, Train: 58.62%, Valid: 64.41% Test: 54.05%
Run: 10, Epoch: 76, Loss: 1.0737, Train: 60.92%, Valid: 61.02% Test: 56.76%
Run: 10, Epo

Run: 10, Epoch: 198, Loss: 0.7515, Train: 73.56%, Valid: 64.41% Test: 54.05%
Run: 10, Epoch: 199, Loss: 0.7682, Train: 73.56%, Valid: 64.41% Test: 59.46%
Run: 10, Epoch: 200, Loss: 0.7820, Train: 73.56%, Valid: 64.41% Test: 59.46%
Run 10:
Highest Train: 77.01
Highest Valid: 74.58
  Final Train: 71.26
   Final Test: 62.16
All runs:
Highest Train: 78.05 ± 3.40
Highest Valid: 64.92 ± 8.03
  Final Train: 66.32 ± 8.06
   Final Test: 53.51 ± 6.60


# TOPO-GAT

In [55]:
dataset = WebKB(root='/tmp/Cornell', name='Cornell',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[183, 1703], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=298])


In [56]:
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

Data(x=[183, 1703], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10], adj_t=[183, 183, nnz=298], topo=[183, 24])


In [60]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 8, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}
    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    model = GAT(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout,args.heads)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 10,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(20, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.01)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.01)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.5720, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 02, Loss: 1.4151, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 03, Loss: 1.3590, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 04, Loss: 1.2601, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 05, Loss: 1.2076, Train: 41.38%, Valid: 52.54% Test: 40.54%
Run: 01, Epoch: 06, Loss: 1.0945, Train: 41.38%, Valid: 52.54% Test: 43.24%
Run: 01, Epoch: 07, Loss: 1.1032, Train: 45.98%, Valid: 52.54% Test: 43.24%
Run: 01, Epoch: 08, Loss: 0.9831, Train: 48.28%, Valid: 52.54% Test: 43.24%
Run: 01, Epoch: 09, Loss: 0.9250, Train: 51.72%, Valid: 52.54% Test: 43.24%
Run: 01, Epoch: 10, Loss: 0.9170, Train: 55.17%, Valid: 52.54% Test: 43.24%
Run: 01, Epoch: 11, Loss: 0.8605, Train: 59.77%, Valid: 52.54% Test: 43.24%
Run: 01, Epoch: 12, Loss: 0.8082, Train: 63.22%, Valid: 54.24% Test: 45.95%
Run: 01, Epoch: 13, Loss: 0.7759, Train: 70.11%, Valid: 50.85% Test: 43.24%
Run: 01, Epo

Run: 01, Epoch: 126, Loss: 0.1627, Train: 98.85%, Valid: 44.07% Test: 37.84%
Run: 01, Epoch: 127, Loss: 0.1981, Train: 98.85%, Valid: 45.76% Test: 37.84%
Run: 01, Epoch: 128, Loss: 0.2756, Train: 98.85%, Valid: 45.76% Test: 37.84%
Run: 01, Epoch: 129, Loss: 0.2005, Train: 98.85%, Valid: 42.37% Test: 37.84%
Run: 01, Epoch: 130, Loss: 0.2315, Train: 98.85%, Valid: 42.37% Test: 37.84%
Run: 01, Epoch: 131, Loss: 0.1592, Train: 98.85%, Valid: 44.07% Test: 37.84%
Run: 01, Epoch: 132, Loss: 0.2397, Train: 98.85%, Valid: 44.07% Test: 43.24%
Run: 01, Epoch: 133, Loss: 0.1303, Train: 98.85%, Valid: 45.76% Test: 45.95%
Run: 01, Epoch: 134, Loss: 0.1948, Train: 98.85%, Valid: 45.76% Test: 43.24%
Run: 01, Epoch: 135, Loss: 0.1397, Train: 98.85%, Valid: 45.76% Test: 45.95%
Run: 01, Epoch: 136, Loss: 0.2571, Train: 98.85%, Valid: 45.76% Test: 45.95%
Run: 01, Epoch: 137, Loss: 0.2279, Train: 98.85%, Valid: 47.46% Test: 45.95%
Run: 01, Epoch: 138, Loss: 0.2765, Train: 98.85%, Valid: 45.76% Test: 45.95%

Run: 02, Epoch: 50, Loss: 0.3294, Train: 95.40%, Valid: 44.07% Test: 40.54%
Run: 02, Epoch: 51, Loss: 0.2278, Train: 96.55%, Valid: 45.76% Test: 37.84%
Run: 02, Epoch: 52, Loss: 0.2867, Train: 98.85%, Valid: 47.46% Test: 35.14%
Run: 02, Epoch: 53, Loss: 0.2843, Train: 98.85%, Valid: 55.93% Test: 35.14%
Run: 02, Epoch: 54, Loss: 0.2202, Train: 98.85%, Valid: 57.63% Test: 35.14%
Run: 02, Epoch: 55, Loss: 0.2874, Train: 97.70%, Valid: 57.63% Test: 37.84%
Run: 02, Epoch: 56, Loss: 0.1759, Train: 97.70%, Valid: 59.32% Test: 37.84%
Run: 02, Epoch: 57, Loss: 0.2201, Train: 97.70%, Valid: 57.63% Test: 37.84%
Run: 02, Epoch: 58, Loss: 0.2026, Train: 98.85%, Valid: 55.93% Test: 37.84%
Run: 02, Epoch: 59, Loss: 0.2051, Train: 100.00%, Valid: 54.24% Test: 35.14%
Run: 02, Epoch: 60, Loss: 0.2175, Train: 100.00%, Valid: 54.24% Test: 35.14%
Run: 02, Epoch: 61, Loss: 0.2783, Train: 100.00%, Valid: 55.93% Test: 37.84%
Run: 02, Epoch: 62, Loss: 0.1905, Train: 100.00%, Valid: 54.24% Test: 35.14%
Run: 02,

Run: 02, Epoch: 177, Loss: 0.1887, Train: 100.00%, Valid: 44.07% Test: 29.73%
Run: 02, Epoch: 178, Loss: 0.0890, Train: 100.00%, Valid: 42.37% Test: 27.03%
Run: 02, Epoch: 179, Loss: 0.1699, Train: 100.00%, Valid: 42.37% Test: 24.32%
Run: 02, Epoch: 180, Loss: 0.0684, Train: 100.00%, Valid: 42.37% Test: 24.32%
Run: 02, Epoch: 181, Loss: 0.1125, Train: 100.00%, Valid: 42.37% Test: 24.32%
Run: 02, Epoch: 182, Loss: 0.1583, Train: 100.00%, Valid: 42.37% Test: 24.32%
Run: 02, Epoch: 183, Loss: 0.2061, Train: 100.00%, Valid: 44.07% Test: 24.32%
Run: 02, Epoch: 184, Loss: 0.1151, Train: 98.85%, Valid: 44.07% Test: 27.03%
Run: 02, Epoch: 185, Loss: 0.1639, Train: 100.00%, Valid: 44.07% Test: 24.32%
Run: 02, Epoch: 186, Loss: 0.0770, Train: 100.00%, Valid: 42.37% Test: 24.32%
Run: 02, Epoch: 187, Loss: 0.1637, Train: 100.00%, Valid: 42.37% Test: 27.03%
Run: 02, Epoch: 188, Loss: 0.1156, Train: 100.00%, Valid: 42.37% Test: 27.03%
Run: 02, Epoch: 189, Loss: 0.0781, Train: 98.85%, Valid: 42.37% T

Run: 03, Epoch: 105, Loss: 0.0737, Train: 98.85%, Valid: 44.07% Test: 54.05%
Run: 03, Epoch: 106, Loss: 0.1294, Train: 98.85%, Valid: 44.07% Test: 54.05%
Run: 03, Epoch: 107, Loss: 0.1237, Train: 97.70%, Valid: 44.07% Test: 54.05%
Run: 03, Epoch: 108, Loss: 0.0915, Train: 97.70%, Valid: 42.37% Test: 54.05%
Run: 03, Epoch: 109, Loss: 0.1029, Train: 100.00%, Valid: 42.37% Test: 54.05%
Run: 03, Epoch: 110, Loss: 0.1652, Train: 100.00%, Valid: 42.37% Test: 54.05%
Run: 03, Epoch: 111, Loss: 0.1761, Train: 100.00%, Valid: 42.37% Test: 54.05%
Run: 03, Epoch: 112, Loss: 0.1236, Train: 100.00%, Valid: 42.37% Test: 54.05%
Run: 03, Epoch: 113, Loss: 0.2020, Train: 100.00%, Valid: 42.37% Test: 54.05%
Run: 03, Epoch: 114, Loss: 0.1098, Train: 100.00%, Valid: 42.37% Test: 51.35%
Run: 03, Epoch: 115, Loss: 0.1299, Train: 100.00%, Valid: 42.37% Test: 54.05%
Run: 03, Epoch: 116, Loss: 0.1034, Train: 100.00%, Valid: 40.68% Test: 54.05%
Run: 03, Epoch: 117, Loss: 0.1488, Train: 98.85%, Valid: 40.68% Test

...

Run: 07, Epoch: 188, Loss: 0.0735, Train: 98.85%, Valid: 35.59% Test: 45.95%
Run: 07, Epoch: 189, Loss: 0.2554, Train: 98.85%, Valid: 33.90% Test: 45.95%
Run: 07, Epoch: 190, Loss: 0.1097, Train: 98.85%, Valid: 33.90% Test: 45.95%
Run: 07, Epoch: 191, Loss: 0.0652, Train: 98.85%, Valid: 33.90% Test: 43.24%
Run: 07, Epoch: 192, Loss: 0.0918, Train: 98.85%, Valid: 33.90% Test: 45.95%
Run: 07, Epoch: 193, Loss: 0.1621, Train: 100.00%, Valid: 33.90% Test: 45.95%
Run: 07, Epoch: 194, Loss: 0.0721, Train: 100.00%, Valid: 35.59% Test: 45.95%
Run: 07, Epoch: 195, Loss: 0.0718, Train: 98.85%, Valid: 37.29% Test: 48.65%
Run: 07, Epoch: 196, Loss: 0.0913, Train: 98.85%, Valid: 37.29% Test: 48.65%
Run: 07, Epoch: 197, Loss: 0.0924, Train: 98.85%, Valid: 37.29% Test: 48.65%
Run: 07, Epoch: 198, Loss: 0.0661, Train: 98.85%, Valid: 37.29% Test: 51.35%
Run: 07, Epoch: 199, Loss: 0.1105, Train: 98.85%, Valid: 35.59% Test: 45.95%
Run: 07, Epoch: 200, Loss: 0.1076, Train: 100.00%, Valid: 37.29% Test: 45.

Run: 08, Epoch: 113, Loss: 0.1488, Train: 100.00%, Valid: 44.07% Test: 43.24%
Run: 08, Epoch: 114, Loss: 0.1954, Train: 100.00%, Valid: 44.07% Test: 43.24%
Run: 08, Epoch: 115, Loss: 0.1284, Train: 100.00%, Valid: 44.07% Test: 40.54%
Run: 08, Epoch: 116, Loss: 0.1482, Train: 100.00%, Valid: 44.07% Test: 40.54%
Run: 08, Epoch: 117, Loss: 0.1434, Train: 100.00%, Valid: 44.07% Test: 40.54%
Run: 08, Epoch: 118, Loss: 0.1744, Train: 100.00%, Valid: 44.07% Test: 43.24%
Run: 08, Epoch: 119, Loss: 0.1988, Train: 100.00%, Valid: 44.07% Test: 43.24%
Run: 08, Epoch: 120, Loss: 0.2654, Train: 100.00%, Valid: 42.37% Test: 43.24%
Run: 08, Epoch: 121, Loss: 0.1977, Train: 100.00%, Valid: 44.07% Test: 43.24%
Run: 08, Epoch: 122, Loss: 0.1617, Train: 100.00%, Valid: 37.29% Test: 43.24%
Run: 08, Epoch: 123, Loss: 0.1646, Train: 100.00%, Valid: 37.29% Test: 40.54%
Run: 08, Epoch: 124, Loss: 0.1250, Train: 100.00%, Valid: 37.29% Test: 40.54%
Run: 08, Epoch: 125, Loss: 0.0897, Train: 100.00%, Valid: 37.29%

Run: 09, Epoch: 39, Loss: 0.3611, Train: 91.95%, Valid: 55.93% Test: 48.65%
Run: 09, Epoch: 40, Loss: 0.3351, Train: 90.80%, Valid: 59.32% Test: 48.65%
Run: 09, Epoch: 41, Loss: 0.4319, Train: 70.11%, Valid: 37.29% Test: 32.43%
Run: 09, Epoch: 42, Loss: 0.3845, Train: 73.56%, Valid: 42.37% Test: 37.84%
Run: 09, Epoch: 43, Loss: 0.5003, Train: 94.25%, Valid: 61.02% Test: 45.95%
Run: 09, Epoch: 44, Loss: 0.4036, Train: 94.25%, Valid: 57.63% Test: 43.24%
Run: 09, Epoch: 45, Loss: 0.3099, Train: 95.40%, Valid: 57.63% Test: 45.95%
Run: 09, Epoch: 46, Loss: 0.3294, Train: 93.10%, Valid: 55.93% Test: 45.95%
Run: 09, Epoch: 47, Loss: 0.2854, Train: 93.10%, Valid: 55.93% Test: 40.54%
Run: 09, Epoch: 48, Loss: 0.3555, Train: 95.40%, Valid: 57.63% Test: 43.24%
Run: 09, Epoch: 49, Loss: 0.3579, Train: 93.10%, Valid: 59.32% Test: 43.24%
Run: 09, Epoch: 50, Loss: 0.3613, Train: 93.10%, Valid: 57.63% Test: 40.54%
Run: 09, Epoch: 51, Loss: 0.2960, Train: 93.10%, Valid: 57.63% Test: 40.54%
Run: 09, Epo

Run: 09, Epoch: 166, Loss: 0.1705, Train: 97.70%, Valid: 57.63% Test: 37.84%
Run: 09, Epoch: 167, Loss: 0.1209, Train: 97.70%, Valid: 57.63% Test: 37.84%
Run: 09, Epoch: 168, Loss: 0.1778, Train: 97.70%, Valid: 57.63% Test: 37.84%
Run: 09, Epoch: 169, Loss: 0.1069, Train: 97.70%, Valid: 57.63% Test: 37.84%
Run: 09, Epoch: 170, Loss: 0.2096, Train: 97.70%, Valid: 55.93% Test: 35.14%
Run: 09, Epoch: 171, Loss: 0.1827, Train: 97.70%, Valid: 57.63% Test: 35.14%
Run: 09, Epoch: 172, Loss: 0.1568, Train: 97.70%, Valid: 54.24% Test: 35.14%
Run: 09, Epoch: 173, Loss: 0.1286, Train: 96.55%, Valid: 54.24% Test: 35.14%
Run: 09, Epoch: 174, Loss: 0.1947, Train: 97.70%, Valid: 54.24% Test: 35.14%
Run: 09, Epoch: 175, Loss: 0.2170, Train: 97.70%, Valid: 54.24% Test: 37.84%
Run: 09, Epoch: 176, Loss: 0.1714, Train: 98.85%, Valid: 54.24% Test: 37.84%
Run: 09, Epoch: 177, Loss: 0.2177, Train: 98.85%, Valid: 55.93% Test: 37.84%
Run: 09, Epoch: 178, Loss: 0.1380, Train: 97.70%, Valid: 55.93% Test: 40.54%

Run: 10, Epoch: 92, Loss: 0.1512, Train: 98.85%, Valid: 44.07% Test: 29.73%
Run: 10, Epoch: 93, Loss: 0.0842, Train: 98.85%, Valid: 44.07% Test: 32.43%
Run: 10, Epoch: 94, Loss: 0.1478, Train: 98.85%, Valid: 44.07% Test: 35.14%
Run: 10, Epoch: 95, Loss: 0.1029, Train: 98.85%, Valid: 44.07% Test: 35.14%
Run: 10, Epoch: 96, Loss: 0.1362, Train: 100.00%, Valid: 44.07% Test: 37.84%
Run: 10, Epoch: 97, Loss: 0.1114, Train: 100.00%, Valid: 44.07% Test: 35.14%
Run: 10, Epoch: 98, Loss: 0.1118, Train: 100.00%, Valid: 47.46% Test: 35.14%
Run: 10, Epoch: 99, Loss: 0.1958, Train: 100.00%, Valid: 47.46% Test: 35.14%
Run: 10, Epoch: 100, Loss: 0.0899, Train: 98.85%, Valid: 47.46% Test: 35.14%
Run: 10, Epoch: 101, Loss: 0.1108, Train: 98.85%, Valid: 47.46% Test: 35.14%
Run: 10, Epoch: 102, Loss: 0.0839, Train: 100.00%, Valid: 42.37% Test: 35.14%
Run: 10, Epoch: 103, Loss: 0.1401, Train: 100.00%, Valid: 40.68% Test: 37.84%
Run: 10, Epoch: 104, Loss: 0.1139, Train: 100.00%, Valid: 40.68% Test: 37.84%
